In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/profpriyanka/PriyankaDas.wav
/kaggle/input/audiofiles/Shourya.wav
/kaggle/input/audiofiles/Aditi.wav
/kaggle/input/audiofiles/Ansh.wav
/kaggle/input/audiofiles/Shaivi.wav
/kaggle/input/audiofiles/Thrissha.wav
/kaggle/input/audiofiles/Kushal.wav
/kaggle/input/audiofiles/Shriya.wav
/kaggle/input/audiofiles/Neha.wav
/kaggle/input/audiofiles/Paurnnamy.wav
/kaggle/input/audiofiles/Siddharth.wav
/kaggle/input/audiofiles/Harsh.wav
/kaggle/input/audiofiles/Yashvi.wav
/kaggle/input/audiofiles/Shive.wav
/kaggle/input/audiofiles/Arjun.wav
/kaggle/input/audiofiles/Vishudha.wav
/kaggle/input/audiofiles/Akshaya.wav
/kaggle/input/audiofiles/Navya.wav
/kaggle/input/audiofiles/Hemakshi.wav
/kaggle/input/audiofiles/Chandrima.wav
/kaggle/input/audiofiles/Ketaki.wav
/kaggle/input/newtest/RaghuramSir.wav
/kaggle/input/harshitaivan/Harshita.wav
/kaggle/input/harshitaivan/Ivan.wav
/kaggle/input/aadityagautam/Aaditya.wav
/kaggle/input/aadityagautam/Aaditya2.wav
/kaggle/input/aadityagautam/Gautam

In [ ]:
name = "user"

**AUDIO TO TEXT**

In [ ]:
audio_file = "/kaggle/input/audiofiles/{}.wav".format(name)
result = model.transcribe(audio_file)
text = result.text
text

Transcribe:   0%|          | 0/236.74 [00:00<?, ?sec/s]

Detected language: english


Transcribe:  99%|█████████▉| 234.76/236.74 [00:36<00:00,  6.42sec/s]


" Okay, the lie detector was asleep when he heard the telephone ringing. At first, he believed it was the clock ordering him to rise, but then he awakened completely and remembered his profession. The voice he heard was rusty, as if disguised. He could not distinguish what altered it, alcohol, drugs, anxiety or fear. It was a woman's voice, but it could have been an adolescent imitating a woman. A woman imitating an adolescent. What is it? he asked. I had to talk to someone. I can't sleep. I had to call someone. You have something to confess? To confess? echoed the voice incredulously. This time, the ascending tonality is unmistakably feminine. Don't you know who I am? No, I just dialed blindly. I've done this before. It is good to hear a voice in the middle of the night, that's all. Why a stranger? You could call a friend. A stranger doesn't ask questions, but it's my profession to ask questions. Who are you? A lie detector. There was long silence after his words. The lie detector exp

**GENERATE EMPHASIZED REGIONS**

In [ ]:
import numpy as np
import librosa
import math

def extract_pitch(audio_file, sr=22000, hop_length=220):
    y, sr = librosa.load(audio_file, sr=sr)
    pitches, magnitudes = librosa.core.piptrack(y=y, sr=sr, hop_length=hop_length)
    pitch_values = []
    for t in range(pitches.shape[1]):
        pitch_slice = pitches[:, t]
        pitch_value = pitch_slice[pitch_slice > 0]
        if len(pitch_value) > 0:
            pitch_values.append(np.max(pitch_value))
        else:
            pitch_values.append(0)
    return np.array(pitch_values)

def pitch_activity(pitch_values, sr=22000, window_size=1, hop_length=220):
    frames_per_second = sr / hop_length
    window_frames = int(window_size * frames_per_second)
    pitch_activity_list = []
    for i in range(0, len(pitch_values), window_frames):
        window = pitch_values[i:i + window_frames]
        if len(window) > 0:
            std_dev = np.std(window)
            pitch_range = np.max(window) - np.min(window)
        else:
            std_dev, pitch_range = 0, 0
        pitch_activity_list.append((std_dev, pitch_range))
    return np.array(pitch_activity_list)


def detect_emphasis(pitch_values):
    thresholds = np.percentile(pitch_values, 98.5)
    emphasized_frames = []
    for i in range (0, len(pitch_values)):
        if pitch_values[i] >= thresholds:
            emphasized_frames.append(i)
    return np.array(emphasized_frames)

def score_windows(emphasized_frames):
    scores = {}
    for i in emphasized_frames:
        if (math.floor(i/100),math.floor(i/100)+1) in scores.keys():
            scores[(math.floor(i/100),math.floor(i/100)+1)] += 1
        else:
            scores[(math.floor(i/100),math.floor(i/100)+1)] = 1
    return scores

def emphasize_speech(audio_file, sr=22000, hop_length=220):
    pitch_values = extract_pitch(audio_file, sr=sr, hop_length=hop_length)
    # print(len(pitch_values))
    emphasized_frames = detect_emphasis(pitch_values)
    # print(emphasized_frames)
    window_scores = score_windows(emphasized_frames)
    return window_scores

regions = emphasize_speech("/kaggle/input/audiofiles/{}.wav".format(name))

In [ ]:
timestamp_ranges=[]
for i in regions.keys():
    timestamp_ranges.append(i)

len(regions)

152

**WORD TIMESTAMP MAPPING**

In [ ]:
import stable_whisper

In [ ]:
result = model.align('/kaggle/input/audiofiles/{}.wav'.format(name), text, language='en') #the alignment step
result.save_as_json("timestamps.json")

Align: 100%|██████████| 236.74/236.74 [00:09<00:00, 24.62sec/s]
Adjustment: 100%|██████████| 234.16/234.16 [00:00<00:00, 7345.84sec/s]


Saved: /kaggle/working/timestamps.json


**HIGHLIGHT EMPHASIZED REGIONS**

In [ ]:
import json

# Load input from timestamps.json
with open('timestamps.json', 'r') as file:
    input_json = json.load(file)

# Save the contents back to a new file with the same data but formatted with indent=4
with open('formatted_timestamps.json', 'w') as outfile:
    json.dump(input_json, outfile, indent=4)

print("The original JSON structure has been reformatted and saved to 'formatted_timestamps.json'.")


# Load input from timestamps.json
with open('timestamps.json', 'r') as file:
    input_json = json.load(file)

# Function to check if a word's timestamp falls in any of the given timestamp ranges
def is_highlighted(word_start, word_end, timestamp_ranges):
    for start, end in timestamp_ranges:
        if word_start < end and word_end > start:
            return True, regions[(start, end)]
    return False, 0

# Process each word in the JSON to add the 'highlight' field
for segment in input_json["segments"]:
    for word in segment["words"]:
        word_start = word["start"]
        word_end = word["end"]
        word["highlight"], word["score"] = is_highlighted(word_start, word_end, timestamp_ranges)

# Save the modified JSON to a new file (highlighted_timestamps.json) with proper formatting
with open('highlighted_timestamps.json', 'w') as outfile:
    json.dump(input_json, outfile, indent=4)

print("The updated JSON with 'highlight' field (both true and false) has been saved to 'highlighted_timestamps.json'.")


The original JSON structure has been reformatted and saved to 'formatted_timestamps.json'.
The updated JSON with 'highlight' field (both true and false) has been saved to 'highlighted_timestamps.json'.


**GENERATE HIGHLIGHTED TEXT**

In [ ]:
import json

def reconstruct_text_from_json(file_path):
    # Open and read the JSON file
    with open(file_path, 'r') as file:
        json_data = json.load(file)

    final_text = []
    current_highlight_group = []

    # Iterate over each word in the segments
    total = 0
    count = 0
    for segment in json_data['segments']:
        for word_data in segment['words']:
            word = word_data['word']
            highlight = word_data['highlight']
            score = word_data['score']
            if highlight:
                # If the word is highlighted, append it to the current highlighted group
                total += score
                count += 1
                current_highlight_group.append(word)
            else:
                # If the word is not highlighted and there are highlighted words collected,
                # finalize the highlighted group and append it to the final text
                if current_highlight_group:
                    final_text.append('{'+ str(total/count) + '{' + ''.join(current_highlight_group) + '}}')
                    current_highlight_group = []
                    total = 0
                    count = 0
                # Append the non-highlighted word directly to the final text
                final_text.append(word)

    # After loop, if there are any leftover highlighted words, append them
    if current_highlight_group:
        final_text.append('{'+ str(total/count) + '{' + ''.join(current_highlight_group) + '}}')

    # Join all parts of the text and return the result
    return ' '.join(final_text)

# Example usage: Assuming the input JSON file is 'highlighted_timestamps.json'
file_path = 'highlighted_timestamps.json'
reconstructed_text = reconstruct_text_from_json(file_path)
print(reconstructed_text)

{1.3{ Okay, the lie detector was asleep when he heard the}}  telephone  ringing.  At  first, {1.0{ he believed it was the}}  clock  ordering  him  to  rise, {1.0{ but then he awakened completely and remembered}}  his  profession. {3.0{ The voice}}  he  heard  was  rusty, {3.0{ as if disguised.}}  He  could  not  distinguish  what  altered  it, {3.0{ alcohol,}}  drugs,  anxiety  or  fear. {1.0{ It was a woman's voice, but}}  it  could  have  been  an  adolescent  imitating  a  woman. {1.0{ A woman imitating}}  an  adolescent. {1.9285714285714286{ What is it? he asked. I had to talk to someone. I can't sleep.}}  I  had  to  call  someone. {3.0{ You}}  have  something  to  confess?  To  confess? {3.0{ echoed}}  the  voice {2.0{ incredulously. This}}  time,  the  ascending  tonality  is  unmistakably  feminine. {2.0{ Don't you know who I am? No, I just dialed}}  blindly. {2.5714285714285716{ I've done this before. It is}}  good  to  hear  a  voice {1.6666666666666667{ in the middle of the 

**PROMPT BASED SUMMARIZATION**

In [ ]:
OPENAI_API_KEY="#################################"

In [ ]:
text_word_count=len(text.split(" "))
text_word_count

612

In [ ]:
import os
from openai import OpenAI
import string

os.environ['OPENAI_API_KEY'] = '#################################'

client = OpenAI(
    api_key = os.getenv('OPENAI_API_KEY'),
)

def count_words(sentence):
    # Remove punctuation from the sentence
    sentence_cleaned = sentence.translate(str.maketrans('', '', string.punctuation))

    # Split the sentence into words
    words = sentence_cleaned.split()

    # Return the number of words
    return len(words)

def summarize_with_emphasis(text):
    prompt = f"""
        I have a piece of text where certain parts are emphasized. The emphasized parts are enclosed between `{{` and `}}`. There is an emphasis score enclosed between the two opening curly braces. Please generate a summary of the text, giving more importance to the emphasized parts with respect to the emphasis scores. Unemphasized text should be considered, but with lower priority. The summary must be approximately half as long as the original text. The summary must be coherent.

        Here is the text:
        {text}

        Summary:
        """

    response = client.completions.create(
        model = "gpt-3.5-turbo-instruct",
        prompt = prompt,
        max_tokens = 1000,
        temperature = 0,
    )

    # Extract and return the generated summary
    summary = response.choices[0].text.strip()
    return summary

# Get the summary
summary1 = summarize_with_emphasis(reconstructed_text)

# Output the summary
print("Summary:", summary1)

Summary: The lie detector was asleep when he heard the telephone ringing. He believed it was the clock ordering him to rise, but then he remembered his profession. The voice he heard was rusty, as if disguised. He could not distinguish what altered it, alcohol, drugs, anxiety or fear. It was a woman's voice, but it could have been an adolescent imitating a woman. A woman imitating an adolescent. "What is it?" he asked. "I had to talk to someone. I can't sleep." "I had to call someone." "You have something to confess?" echoed the voice incredulously. "This time, the ascending tonality is unmistakably feminine." "Don't you know who I am?" "No, I just dialed blindly." "I've done this before. It is good to hear a voice in the middle of the night, that's all." "Why a stranger? You could call a friend. A stranger doesn't ask questions, but it's my profession to ask questions." "Who are you?" "A lie detector." There was long silence after his words. The lie detector expected her to hang up, b

In [ ]:
x = summary1.split(" ")
len(x)

601

In [ ]:
import os
from openai import OpenAI
import string

os.environ['OPENAI_API_KEY'] = '#################################'

client = OpenAI(
    api_key = os.getenv('OPENAI_API_KEY'),
)

def count_words(sentence):
    # Remove punctuation from the sentence
    sentence_cleaned = sentence.translate(str.maketrans('', '', string.punctuation))

    # Split the sentence into words
    words = sentence_cleaned.split()

    # Return the number of words
    return len(words)

def summarize(text):
    prompt = f"""
        I have a piece of text. Please generate a summary of the text. The summary must be approximately half as long as the original text. The summary must be coherent.

        Text:
        {text}

        Summary:
        """

    response = client.completions.create(
        model = "gpt-3.5-turbo-instruct",
        prompt = prompt,
        max_tokens = 2000,
        temperature = 0,
    )

    # Extract and return the generated summary
    summary = response.choices[0].text.strip()
    return summary

summary2 = summarize(text)

# Output the summary
print("Summary:", summary2)

Summary: A lie detector receives a call from a woman who is unable to sleep and needs to talk to someone. The lie detector tries to figure out who she is and why she called. They discuss the concept of guilt and confessing to crimes. The woman hangs up and the lie detector traces the call to a bar where he sees a woman dressed in red and silver. She evokes strong emotions in him and he sees her as a ladder to climb towards love and poetry.


In [ ]:
x = summary2.split(" ")
len(x)

87

In [ ]:
import random
variables = [summary1, summary2]

# Shuffle the list randomly
random.shuffle(variables)

# Print the variables in the randomized order
for i in variables:
    print(i)
    print(" ")

A lie detector receives a call from a woman who is unable to sleep and needs to talk to someone. The lie detector tries to figure out who she is and why she called. They discuss the concept of guilt and confessing to crimes. The woman hangs up and the lie detector traces the call to a bar where he sees a woman dressed in red and silver. She evokes strong emotions in him and he sees her as a ladder to climb towards love and poetry.
 
The lie detector was asleep when he heard the telephone ringing. He believed it was the clock ordering him to rise, but then he remembered his profession. The voice he heard was rusty, as if disguised. He could not distinguish what altered it, alcohol, drugs, anxiety or fear. It was a woman's voice, but it could have been an adolescent imitating a woman. A woman imitating an adolescent. "What is it?" he asked. "I had to talk to someone. I can't sleep." "I had to call someone." "You have something to confess?" echoed the voice incredulously. "This time, the 